In [1]:
# imports
import pandas as pd
import requests
import os
from urllib3.exceptions import IncompleteRead


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [9]:
salvador_station = pd.read_csv(r"C:\Users\johnk\OneDrive\Desktop\Statistical-Modelling-Project\salvador_station.csv")

foursquare_url = "https://api.foursquare.com/v3/places/search"
API_key =os.getenv("4square_api_key")

if not API_key:
    print("API key not found. Please set the FOURSQUARE_API_KEY environment variable.")
    exit()
headers = {
    "Accept": "application/json",
    "Authorization": API_key
}


business_terms = {
    "13003": "Bars",
    "13065": "Restaurants",
    "18000": "Sports & Recreation"}

foursquare_result = []

for _, row in salvador_station.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    for term, category_name in business_terms.items():
        params ={
            "ll": f"{latitude},{longitude}",
            "radius": 1000,
            "limit": 50,
            "categories": term}
        
        try:
            response = requests.get (foursquare_url,headers = headers, params = params, timeout=300)

            if response.status_code == 200:
                data = response.json()
                venues = data.get("results", [])
                for venue in data.get("results", []):
                    foursquare_result.append({
                        "Station Name": row['Station Name'],
                        "Venue Name": venue.get('name', 'N/A'),
                        "Category": category_name,
                        "Venue Latitude": venue.get('geocodes', {}).get('main', {}).get('latitude'),
                        "Venue Longitude": venue.get('geocodes', {}).get('main', {}).get('longitude'),
                        "fsq_id": venue.get('fsq_id', 'N/A'),
                        "venue distance": venue.get('distance')
                        })
            else:
                print(f"Error for location {latitude},{longitude}: {response.status_code}")
        except IncompleteRead as e:
            print(f"Error reading response for {latitude},{longitude}: {e}")
        except Exception as e:
            print(f"An error occurred for {latitude},{longitude}: {e}") 


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [12]:
foursquare_df = pd.DataFrame(foursquare_result)

In [13]:
foursquare_df

,Station Name,Venue Name,Category,Venue Latitude,Venue Longitude,fsq_id,venue distance
0,27 - Praça Ana Lúcia Magalhães,Preto,Bars,-12.991484,-38.460663,4c4a3bd5959220a13290d10d,207
1,27 - Praça Ana Lúcia Magalhães,Proa Cervejaria Salvador,Bars,-12.994175,-38.457994,5ce8a0eb1acf11002cdee3a2,326
2,27 - Praça Ana Lúcia Magalhães,Su Espeto Bistrot,Bars,-12.993303,-38.460751,4fceabcce4b0296cfc3c8836,16
3,27 - Praça Ana Lúcia Magalhães,Restaurante Caminho de Casa,Bars,-12.995514,-38.464786,4c0da9b97189c928fb6dd7b6,479
4,27 - Praça Ana Lúcia Magalhães,Acqua Café,Bars,-12.993441,-38.460507,504fcb01e4b0eec17d50ff2b,38
...,...,...,...,...,...,...,...
7712,29 - Praça do Sol,Academia Rede Bahia,Sports & Recreation,-13.003751,-38.512999,53ece8de498e7f61e52d47b9,800
7713,29 - Praça do Sol,XIX Eref,Sports & Recreation,-13.002363,-38.509099,4ffe38cde4b0eb8e42e40170,889
7714,29 - Praça do Sol,Academia Sbelt,Sports & Recreation,-13.006691,-38.502082,579274f9498e9b4271f8157d,944
7715,29 - Praça do Sol,"Rádio Facom 89,3 FM",Sports & Recreation,-13.001499,-38.510380,4d0377659b7f60fc739a08ab,980


In [11]:
foursquare_df.to_csv('foursquare_data.csv',index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
salvadore_station= pd.read_csv(r"C:\Users\johnk\OneDrive\Desktop\Statistical-Modelling-Project\salvador_station.csv")




yelp_url = "https://api.yelp.com/v3/businesses/search"
# Your API Key
api_key = os.getenv("yelp_api_key")

if not api_key:
    print("API key not found. Please set the yelp_API_KEY environment variable.")
    exit()

# Headers with Authorization
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}
yelp_result= []
business_terms = ["Restaurants","Bars","Fitness & Instruction"]


for _, row in salvadore_station.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    for term in business_terms:
# Parameters for Search
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "radius": 1000,  # Radius in meters
            "term": term, 
            "limit": 50 # Number of results to return
}

# Make the Request
        response = requests.get(yelp_url, headers=headers, params=params)

# Check the Response
        if response.status_code == 200:
            data = response.json()
            for business in data.get("businesses", []):
                yelp_result.append({
                    "Station Name": row['Station Name'],
                    "Latitude": latitude,
                    "Longitude": longitude,
                    "Search Term": term,
                    "Business Name": business['name'],
                    "Business Category": term,  # Add term as category
                    "Business Rating": business['rating'],
                    "Business Address": business['location'].get('address1', 'N/A'),
                    "Business Phone": business.get('phone', 'N/A'),
                    "Business Distance (meters)": business.get('distance','N/A')})
        else:
            print(f"Error: {response.status_code}")
            print(f"Response details: {response.json()}")


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [4]:
yelp_df = pd.DataFrame(yelp_result)

In [14]:
yelp_df

,Station Name,Latitude,Longitude,Search Term,Business Name,Business Category,Business Rating,Business Address,Business Phone,Business Distance (meters)
0,27 - Praça Ana Lúcia Magalhães,-12.993338,-38.460908,Restaurants,Mamma's House,Restaurants,4.9,"R. das Margaridas,150",+557133532600,527.483624
1,27 - Praça Ana Lúcia Magalhães,-12.993338,-38.460908,Restaurants,Acqua Café,Restaurants,4.3,"R. das Hortências, 600 - Pituba",+557133581635,47.248643
2,27 - Praça Ana Lúcia Magalhães,-12.993338,-38.460908,Restaurants,Riz Bistrot & Risottos,Restaurants,3.9,"R. das Hortênsias, 966",+557130196800,328.793942
3,27 - Praça Ana Lúcia Magalhães,-12.993338,-38.460908,Restaurants,Vignoli,Restaurants,4.4,"Av. Paulo VI, 1659",+557130182111,490.866547
4,27 - Praça Ana Lúcia Magalhães,-12.993338,-38.460908,Restaurants,Adam Cozinha Originária,Restaurants,4.6,"R. Edith Mendes da Gama e Abreu, 189",+557135063191,208.535935
...,...,...,...,...,...,...,...,...,...,...
6428,29 - Praça do Sol,-13.010308,-38.509888,Fitness & Instruction,Forma A3,Fitness & Instruction,0.0,"Rua Professor Severo Pessoa, n 29, São Lázaro/...",,1334.557276
6429,29 - Praça do Sol,-13.010308,-38.509888,Fitness & Instruction,Active Pilates,Fitness & Instruction,0.0,"Travessa Bartolhomeu Gusmão, 104, Loja L",+557130356511,1392.229543
6430,29 - Praça do Sol,-13.010308,-38.509888,Fitness & Instruction,Triathlon Master,Fitness & Instruction,0.0,Al Rio de São Pedro 12,+557133316134,1680.155173
6431,29 - Praça do Sol,-13.010308,-38.509888,Fitness & Instruction,Life Style Fight Center,Fitness & Instruction,0.0,"Lgo. do Campo Santo, 5",+557191608776,1448.276566


In [8]:
yelp_df.to_csv('yelp_data.csv',index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [15]:
yelp_df_poi_count = len(yelp_df)
foursquare_df_poi_count = len(foursquare_df)

print(f"the number of POI in yelp are {yelp_df_poi_count}")
print(f"the number of POI in foursquare are {foursquare_df_poi_count}")

if yelp_df_poi_count > foursquare_df_poi_count:
    print(f"the yelp DataFrame have more POIs.")
elif foursquare_df_poi_count > yelp_df_poi_count:
    print (f"the foursquare DataFrame have more POIs.")


the number of POI in yelp are 6433
the number of POI in foursquare are 7717
the foursquare DataFrame have more POIs.


In [18]:
missing_data_yelp_df = yelp_df.isnull().sum()
missing_data_foursquare_df = foursquare_df.isnull().sum()

total_missing_data_yelp_df = missing_data_yelp_df.sum()
total_missing_data_foursquare_df = missing_data_foursquare_df.sum()

if total_missing_data_yelp_df > total_missing_data_foursquare_df:
   print(f"The yelp DataFrame have more missing values.")
elif total_missing_data_foursquare_df > total_missing_data_yelp_df:
    print(f"The foursquere DataFrame have more missing values.")

The yelp DataFrame have more missing values.


Get the top 10 restaurants according to their rating

In [19]:
yelp_df.sort_values(by="Business Rating", ascending = False).head(10)

,Station Name,Latitude,Longitude,Search Term,Business Name,Business Category,Business Rating,Business Address,Business Phone,Business Distance (meters)
2199,21 - Campo Grande,-12.989268,-38.520369,Restaurants,Porto do Moreira,Restaurants,5.0,"Rua Carlos Gomes, 486, Campo Grande",+557133224112,974.498151
1999,33 - Jardim de Alah,-12.994033,-38.440247,Restaurants,Spaghetti de Ouro,Restaurants,5.0,"R. Hélio Machado, 14",+557134845958,1426.222813
5499,7 - UFBA Portão 01,-13.002495,-38.507178,Fitness & Instruction,Triathlon Health Club,Fitness & Instruction,5.0,R. Comendador Horácio Urpia Júnior 75,+557133316568,1393.063205
5500,7 - UFBA Portão 01,-13.002495,-38.507178,Fitness & Instruction,Acal Associação Capoeira Arte & Luta,Fitness & Instruction,5.0,"R. Macapá, 474",+557132353601,470.453225
2043,33 - Jardim de Alah,-12.994033,-38.440247,Fitness & Instruction,Corpo Livre Academia,Fitness & Instruction,5.0,Travessa Stiep,+557133425463,1201.503204
2041,33 - Jardim de Alah,-12.994033,-38.440247,Fitness & Instruction,Club Atmos Fitness,Fitness & Instruction,5.0,"R. Alfredo Gomes de Oliveira, 8",+557130365996,762.339315
2037,33 - Jardim de Alah,-12.994033,-38.440247,Fitness & Instruction,Qualitus Saúde,Fitness & Instruction,5.0,"Av. Otávio Mangabeira. nº 4099, Jardim Armação",+557132320023,300.480237
2032,33 - Jardim de Alah,-12.994033,-38.440247,Bars,Bar do Peixoto,Bars,5.0,"R. Gen. Oriosvaldo Pereira Lima, 51",+557133417899,1441.575846
1995,33 - Jardim de Alah,-12.994033,-38.440247,Restaurants,Pizza Hut,Restaurants,5.0,"R. Aristídes Fraga Lima, 175",,1503.992278
272,20 - Metrô Lapa,-12.981050,-38.512030,Restaurants,Encontro dos Artistas,Restaurants,5.0,"Rua das Laranjeiras 15, Pelourinho",+557133211721,862.708893
